### Triadic closeness
In a network a triad is a combination of three nodes, $x$, $z$ and $y$, along with the possible links between them. Every combination is a specific type of triad and for a directed graph there are 16 different ways, but this increases to 36 when we consider that $z$ is a common neighbor to $x$ and $y$ and that for link prediction the order matters, the triad consisting of the links ${(x,y), \ (x,z), \ (y,z)}$ is not the same as  ${(y,x), \ (x,z), \ (y,z)}$. The different combinations are seen in the figure where they are classified based on the link between $x$ and $y$, with 1-9 meaning no link, 11-19 meaning a link from $x$ to $y$, 21-29 meaning a link from $y$ to $x$ and 31-39 being a reciprocal link between $x$ and $y$.

<figure style="display: inline-block; text-align: center; max-width: 100%; margin: 0 auto">
<img src="Pictures/open triads.png" style="margin: 0 auto; max-width: 85%; margin-bottom: 10px">
<figcaption style="font-style: italic">Possible open triadic configurations for a directed graph, with the category label below the triad.</figcaption>
</figure>
<figure style="display: inline-block; float: right; text-align: center; max-width: 45%; margin: 0 20px">
<img src="Pictures/closed triads.png" style="float: right; max-width: 100%; margin-bottom: 10px">
<figcaption style="font-style: italic">Possible closed triadic configurations for a directed graph, with the category label below the triad.</figcaption>
</figure>
Triadic closeness is an unsupervised method proposed by Schall [](#cite-schallTC) that exploits the fact that some triadic patterns are more likely to appear in a given network than others, so if you have a possible link between nodes $x$ and $y$ that have the common neighbor $z$ the likelihood of the created triad appearing in the network can be used as a score. In other words, the likelihood for a link between $x$ and $y$ depends on how common the triad formed from adding the link is compared to how common the triad formed by the unconnected $x$ and $y$.

This is related to [motif analysis](https://en.wikipedia.org/wiki/Network_motif), specifically motif analysis where every motif is a sub-graph of size 3 and Schall [](#cite-schallTC) produced good results on data from Twitter, GitHub and Google+.

The actual triadic closeness score is calculated as

$$
s_{x,y} = \sum_{z \ \in \Gamma(x) \cup \Gamma(y)} \frac{\text{F}(\delta(x,y,z) + 10) + \text{F}(\delta(x,y,z) + 30)}{\text{F}(\delta(x,y,z))}
$$

where $s_{x,y}$ is the score for the $x,y$ node pair, $\Gamma(x)$ returns the neighborhood of $x$, $\delta(x,z,y)$ returns the triad produced by the nodes $x$, $y$ and $z$ and $\text{F}(t)$ returns the frequency of the triad $t$. Note that the neighborhood consists of both in- and out-going links.

Since a triad requires that the nodes $x$ and $y$ are joined through an extra node $z$ nodes that are linked but do not share neighbors will not be found by this method. If the network being examined has important structure beyond the 1-neighborhood more general motif analysis methods can be used.

### Common Referrers
Common refferers is based on the idea that if a $x$ links to $y$ and $z$ also links to $y$ then there is a higher probability of $x$ linking to the other nodes that $z$ links to.


### Results

Performing each index on the full set of real edges and all edges that are known to score higher than 0 on that particular index

||Precision (8700)|Expected for random choice|
|--|--|
|Triadic closeness|0.848|0.104|
|Common neighbors|0.576|0.104|
|Common referrers|0.062|0.021|

To get a more realistic evaluation of the methods each node had all but one of its links removed and the removed links were then predicted along with all other possible links for that node that would score higher than 0. The removed links are then added one by one and evaluated. This gives the results seen in the figure below. Note that while Common referrers still does very badly, triadic closeness no longer outperforms common neighbors. 

<figure style="display: inline-block; text-align: center; margin: 0 20px">
<img src="Pictures/per_node_validation.png" style="max-width: 100%; margin-bottom: 10px">
</figure>

#### Implementation

Implementing triadic closeness scoring requires three parts

* A method that takes three nodes and returns the triad ID
* A method that returns the distribution of triads for an entire network
* A method that calculates the actual score for a pair of nodes, $x$ and $y$

Finding the triad ID is done through the fairly complicated nest of `if` statements below. Note that Python does not have `switch`statements, otherwise this would be the obvious thing to use here.

Returning the triad distribution for the entire network can now be done easily with a nested loop as below. Note that the way Schall set up the different categories of triads means that some information will be duplicated, i.e. every time there is an instance of node $x$ connecting to node $y$ through $z$ the opposite will also be true. `networkx` implements a different method called `nx.triadic_census` which does not do this. Calculating the distribution is the most computationally demanding part of this method, but luckily the results can be stored so it only needs to be computed once for each graph.

Finally the scoring is implemented as a method that takes a candidate edge as an argument and returns that edge with the TC score as an attribute.

These methods together can now be combined with a k-fold validation scheme that works on training and validation sets. If you do this it is important to recalculate the triad distribution at each fold, as changing the training set also means changing the graph.



#### Potential theory

#### Motif analysis

### A common neighbours formulation for the temporal DAG
The directed, acyclic nature can be helpful in establishing a directed version of the undirected common neighborhood of two nodes. For an undirected network like the Facebook friend network the number of friends in common can be a good predictor for a missing or future link, but for a directed and dynamic network like Wikipedia the direction of the links needs to be taken into account. If for instance two articles have many outgoing links in common, i.e. both reference many of the same articles, then it doesn't necessarily mean that these two articles need to be linked. The same goes for the incoming links, if two articles are referenced by many of the same articles it does not necessarily mean that there should be a link between the two. A more logical idea is that if two articles are in the out-neighborhood of many of the same articles, then they should probably be linked

* Common neighbors in the out-neighborhood between $x$ and $y$ indicates that both articles reference the same articles
* Common neighbors in the in-neighborhood between $x$ and $y$ indicates that both articles are referenced by the same articles
* Having $y$ in the out-neighborhood of $z$ and $z$ in the out-neighborhood of $x$ indicates that $y$ is referenced by articles referenced by $x$
* Having $y$ in the in-neighborhood of $z$ and $z$ in the in-neighborhood of $x$ indicates that $y$ references articles that reference $x$

The additional constraint of nodes arriving in the network only having outgoing edges (since they are the newest verdict and thus cannot be referenced by others) leaves us with only two permutations to be considered, which can be seen in the figure below where the possible arrangements are A and B marked with a red border.


It is clear that the simply collapsing these two cases into one and counting the common neighbors is the same as looking at the undirected graph.
Calling the in-neighborhood of the node $v$ for $\Gamma_{in}(v)$ and the out-neighborhood for $\Gamma_{out}(v)$ and assuming that $v$ is a new arrival to the network the following holds

$$
\Gamma(v) = \Gamma_{out}(v) \\
\Gamma(v) \cap \Gamma(u) = (\Gamma_{out}(v) \cap \Gamma_{in}(u)) \cup (\Gamma_{out}(v) \cap \Gamma_{out}(u))
$$

This opens up for the possibility of weighting each component of the neighborhood differently or using different structural methods on each, i.e. Jaccard similarity on incoming edges and Resource Allocation on outgoing edges. 